In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import chemprop

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_lo_metrics
from utils import chemprop_prepare_df, chemprop_process_folder

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/lo/kcnh2/train_1.csv')
test = pd.read_csv('../../../../data/lo/kcnh2/test_1.csv')

train

,Unnamed: 0,smiles,value,cluster
0,0,Brc1ccc2c(NC3=NC[C@@]4(CN5CCC4CC5)O3)ncnn12,5.601886,0
1,1,Brc1cnc2nc(N3CCN4CCC3CC4)oc2c1,5.638083,0
2,2,C#CCOc1cnc(C(=O)Nc2cc(F)c(F)c([C@@]3(C)N=C(N)S...,5.161088,0
3,3,C#Cc1cnc(Nc2cnc(C#N)c(O[C@H](C)CN(C)C)n2)cc1NC,5.096856,0
4,4,C#Cc1cnc(Nc2cnc(C#N)cn2)cc1NC[C@@H]1CNCCO1,5.086133,0
...,...,...,...,...
3308,3308,c1cnc2c(N3CCN(CCCCc4ccc(OCCCN5CCCCCC5)cc4)CC3)...,5.799727,0
3309,3309,c1cnc2c(N3CCN(CCCc4ccc(OCCCN5CCCCCC5)cc4)CC3)c...,5.999566,0
3310,3310,c1cnc2c(N3CCN(CCc4ccc(OCCCN5CCCCCC5)cc4)CC3)cc...,5.099945,0
3311,3311,c1cncc(-c2ccc(-c3noc(C4CN5CCC4CC5)n3)o2)c1,5.193752,0


In [4]:
import os
import os
import shutil

temp_datapath = '/tmp/chemprop/'
if os.path.exists(temp_datapath) and os.path.isdir(temp_datapath):
    shutil.rmtree(temp_datapath)
os.mkdir(temp_datapath)

input_paths = [
    '/data/steshin/gero_benchmark/data/lo/kcnh2/'
]

output_paths = [
    temp_datapath
]

for i in range(len(input_paths)):
    chemprop_process_folder(input_paths[i], output_paths[i])

In [5]:
base_arguments = [
    '--dataset_type', 'regression', 
    '--data_path', temp_datapath + 'train_1.csv',
    '--separate_val_path', temp_datapath + 'test_1.csv',
    '--separate_test_path', temp_datapath + 'test_1.csv',
    '--save_preds',
    '--metric', 'mae',
    '--epochs', '100',
    '--gpu', '1',
    '--save_dir', temp_datapath + 'checkpoint',
    '--features_generator', 'rdkit_2d_normalized',
    '--no_features_scaling',
]

In [6]:
param_dict = {
    '--depth': ['3', '4', '5', '6'],
    '--dropout': ['0.0', '0.2', '0.3', '0.5', '0.7'],
    '--ffn_hidden_size': ['600', '1200', '2400', '3600'],
    '--ffn_num_layers': ['1', '2', '3'],
    '--hidden_size': ['600', '1200', '2400', '3600']
}

In [7]:
from copy import copy
from tqdm import tqdm
from sklearn.model_selection import ParameterSampler

hyperparam_list = list(ParameterSampler(param_dict, n_iter=20))

best_score = 0.0
best_hyperparams = None

for i, hyperparam in tqdm(enumerate(hyperparam_list)):
    print('HYPERPARAMETER', i)
    print(hyperparam)
    result_arguments = copy(base_arguments)
    for key, value in hyperparam.items():
        result_arguments.append(key)
        result_arguments.append(value)
    # Train
    args = chemprop.args.TrainArgs().parse_args(result_arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    test_preds = pd.read_csv(temp_datapath + 'checkpoint/test_preds.csv')
    metrics = get_lo_metrics(test, test_preds['targets'].to_list())

    score = metrics['spearman']
    if score > best_score:
        print('New best spearman:', score)
        best_score = score
        best_hyperparams = hyperparam

0it [00:00, ?it/s]Command line
python /home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9048 --control=9046 --hb=9045 --Session.signature_scheme="hmac-sha256" --Session.key=b"cfd6dacd-c78d-4cc6-b943-501411b0ba98" --shell=9047 --transport="tcp" --iopub=9049 --f=/home/steshin/.local/share/jupyter/runtime/kernel-v2-2545447jAGiwfyW4bgc.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 

HYPERPARAMETER 0
{'--hidden_size': '2400', '--ffn_num_layers': '1', '--ffn_hidden_size': '3600', '--dropout': '0.7', '--depth': '4'}


3313it [00:00, 370029.80it/s]
100%|██████████| 3313/3313 [00:00<00:00, 158594.46it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
406it [00:00, 273908.22it/s]
100%|██████████| 406/406 [00:00<00:00, 156673.79it/s]
406it [00:00, 356178.08it/s]
100%|██████████| 406/406 [00:00<00:00, 156342.95it/s]
Total size = 3,313 | train size = 3,313 | val size = 406 | test size = 406
Fitting scaler
Building model 0
MoleculeModel(
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.7, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=2400, bias=False)
        (W_h): Linear(in_features=2400, out_features=2400, bias=False)
        (W_o): Linear(in_features=2533, out_features=2400, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.7, inplace=False)
    (1): Linear(in_features=2600, out_features=1, bias=True)
  )
)
Number of parameters = 12,199,401
Moving model to cuda
Epoch 0





New best spearman: 0.40664635578239666
HYPERPARAMETER 1
{'--hidden_size': '600', '--ffn_num_layers': '3', '--ffn_hidden_size': '3600', '--dropout': '0.7', '--depth': '3'}


Command line
python /home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9048 --control=9046 --hb=9045 --Session.signature_scheme="hmac-sha256" --Session.key=b"cfd6dacd-c78d-4cc6-b943-501411b0ba98" --shell=9047 --transport="tcp" --iopub=9049 --f=/home/steshin/.local/share/jupyter/runtime/kernel-v2-2545447jAGiwfyW4bgc.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_di

...

3313it [00:00, 352138.29it/s]
100%|██████████| 3313/3313 [00:00<00:00, 156045.88it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
406it [00:00, 250093.61it/s]
100%|██████████| 406/406 [00:00<00:00, 155785.15it/s]
406it [00:00, 317940.15it/s]
100%|██████████| 406/406 [00:00<00:00, 156271.21it/s]
Total size = 3,313 | train size = 3,313 | val size = 406 | test size = 406
Fitting scaler
Building model 0
MoleculeModel(
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.7, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=1200, bias=False)
        (W_h): Linear(in_features=1200, out_features=1200, bias=False)
        (W_o): Linear(in_features=1333, out_features=1200, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.7, inplace=False)
    (1): Linear(in_features=1400, out_features=3600, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.7, inplace=False)
    (4): Linea

HYPERPARAMETER 18
{'--hidden_size': '3600', '--ffn_num_layers': '1', '--ffn_hidden_size': '1200', '--dropout': '0.0', '--depth': '3'}


3313it [00:00, 349991.92it/s]
100%|██████████| 3313/3313 [00:00<00:00, 156923.46it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
406it [00:00, 257117.23it/s]
100%|██████████| 406/406 [00:00<00:00, 155870.70it/s]
406it [00:00, 275458.98it/s]
100%|██████████| 406/406 [00:00<00:00, 153192.46it/s]
Total size = 3,313 | train size = 3,313 | val size = 406 | test size = 406
Fitting scaler
Building model 0
MoleculeModel(
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.0, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=3600, bias=False)
        (W_h): Linear(in_features=3600, out_features=3600, bias=False)
        (W_o): Linear(in_features=3733, out_features=3600, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=3800, out_features=1, bias=True)
  )
)
Number of parameters = 26,939,001
Moving model to cuda
Epoch 0





HYPERPARAMETER 19
{'--hidden_size': '3600', '--ffn_num_layers': '1', '--ffn_hidden_size': '3600', '--dropout': '0.2', '--depth': '4'}


3313it [00:00, 354040.34it/s]
100%|██████████| 3313/3313 [00:00<00:00, 158946.39it/s]
Number of tasks = 1
Fold 0
Splitting data with seed 0
406it [00:00, 292040.37it/s]
100%|██████████| 406/406 [00:00<00:00, 150671.33it/s]
406it [00:00, 377831.69it/s]
100%|██████████| 406/406 [00:00<00:00, 149428.52it/s]
Total size = 3,313 | train size = 3,313 | val size = 406 | test size = 406
Fitting scaler
Building model 0
MoleculeModel(
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.2, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=3600, bias=False)
        (W_h): Linear(in_features=3600, out_features=3600, bias=False)
        (W_o): Linear(in_features=3733, out_features=3600, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=3800, out_features=1, bias=True)
  )
)
Number of parameters = 26,939,001
Moving model to cuda
Epoch 0





In [8]:
print(best_hyperparams)

{'--hidden_size': '3600', '--ffn_num_layers': '2', '--ffn_hidden_size': '1200', '--dropout': '0.7', '--depth': '6'}


In [9]:
print(best_score)

0.4590026202386252


In [10]:
from copy import copy

result_arguments = copy(base_arguments)
result_arguments.extend([
    '--hidden_size', '3600',
    '--ffn_num_layers', '2',
    '--ffn_hidden_size', '1200',
    '--dropout', '0.7',
    '--depth', '6'
])

args = chemprop.args.TrainArgs().parse_args(result_arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)


Command line
python /home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9048 --control=9046 --hb=9045 --Session.signature_scheme="hmac-sha256" --Session.key=b"cfd6dacd-c78d-4cc6-b943-501411b0ba98" --shell=9047 --transport="tcp" --iopub=9049 --f=/home/steshin/.local/share/jupyter/runtime/kernel-v2-2545447jAGiwfyW4bgc.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_di

In [11]:
from copy import copy

max_epoch = 90



Command line
python /home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"6c604d94-190d-4da2-9c36-788708df5377" --shell=9012 --transport="tcp" --iopub=9014 --f=/home/steshin/.local/share/jupyter/runtime/kernel-v2-2822374BRuswu5zWdXq.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_di

In [15]:
get_hi_metrics(test_result,test_result['preds'])

ValueError: Input contains NaN.

In [24]:
test_result

,Unnamed: 0,smiles,value,preds
0,383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True,0.148691
1,386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True,0.137544
2,389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True,0.049242
3,2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True,0.029785
4,2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False,0.523535
...,...,...,...,...
1186,5766,O=S(=O)(c1ccccc1)c1cnc2c(N3CCNCC3)cccc2c1,True,0.073102
1187,6025,O[C@H]1[C@@H](NC[C@@H]2COc3ccccc3O2)CC[C@@H]1O...,False,0.044055
1188,6018,O[C@@H]1[C@H](NC[C@@H]2COc3ccccc3O2)CC[C@H]1Oc...,False,0.737260
1189,6019,O[C@@H]1[C@H](NC[C@H]2COc3ccccc3O2)CC[C@H]1Oc1...,False,0.058900


In [23]:
test

,Unnamed: 0,smiles,value
0,383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True
1,386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True
2,389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True
3,2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True
4,2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False
...,...,...,...
1186,5766,O=S(=O)(c1ccccc1)c1cnc2c(N3CCNCC3)cccc2c1,True
1187,6025,O[C@H]1[C@@H](NC[C@@H]2COc3ccccc3O2)CC[C@@H]1O...,False
1188,6018,O[C@@H]1[C@H](NC[C@@H]2COc3ccccc3O2)CC[C@H]1Oc...,False
1189,6019,O[C@@H]1[C@H](NC[C@H]2COc3ccccc3O2)CC[C@H]1Oc1...,False
